# Inlet Profiles - Exercise

## Exercise 1

Create the following flow profile:

```{figure} ./resources/2_comp.png
:width: 50%
:align: center
```
Assume a flow rate of $Q = 1\cdot mL\cdot min^{-1}$

***Hint:*** We need to specify a second component for the template function and then pass a list of to the concentration parameter.

## Exercise 2

Create the following inlet profile:

```{figure} ./resources/gradient.png
:width: 50%
:align: center
```

Assume a flow rate of $Q = 1\cdot mL\cdot min^{-1}$

***Hint:*** We need three `Events` and for linear gradients, the concentration for each component needs to be passed as a list of cubic polynomial coefficients.

## Bonus Exercise: Changing the system connectivity

Instead of creating inlet profiles by modifying the concentration profile of a single `Inlet` unit operation, the same effect can also be achieved by assuming multiple `Inlets`, each with a constant concentration and changing the flow rates of the system.

***Task:*** Creating a system with a second `Inlet` and recreate the previous examples.

### Example 1

### Example 2